In [33]:
#L'objectif ici est de faire la fusion de nos deux précédents programmes. Celui de Jomuel et le mien.
#A chaque fois on compare l'option chosie par els deux en partant de la structure du mien. 
#On a besoin d'être dans ce fichier afin d'accéder à notre dataset. 

In [34]:
#b. importations
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [35]:
#a. Récupération des datasets
dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")

C:\Users\USER\AppData\Local\Temp\ipykernel_12220\1833793343.py:2: DtypeWarning: Columns (36,38,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")


In [ ]:
#c. Feature Engineering
#Rien d'implémenter de connu pour faire ce que l'on doit faire. 
#Le gros du travail qui va être fait: 

#Eléments à supprimer: 

#On va enlever les éléments qui ont des cases  vides dans: 
#C'est que ce sont des logements saisonniers/temporaires dans une zone de l'alaska sous sous peuplé.
#Que 75000 habitants sur plusieurs millions. 
#Sinon on perd des données c'est dommage et si on les remplace par des 0 cela risque de mal influencer notre modèle. 
#Car on perd plein d'autres données. 
#On enlève donc : ceux qui n'ont pas de county_code 
#et ceux qui n'ont pas:   applicant_income_000s
#On doit supprimer les éléments qui ont vide pour : applicant_income_000s et county_code 
#On crée L liste des éléments à supprimer:
L=[]
#pour county_code
#on va créer un dataset composé uniquement de county_code
#Basée sur la colonne "county_code": 
df_county = pd.DataFrame({"dataset county code":dataset["county_code"]})
#on va en récupérer toutes les lignes nulles
l_nulles_county = df_county.isnull().values.any()
#et les ajouter à L
L+=l_nulles_county
#pour applicant_income_000s
#on va créer un dataset composé uniquement de applicant_income_000s
#Basée sur la colonne "applicant_income_000s": 
df_income = pd.DataFrame({"dataset applicant_income_000s":dataset["applicant_income_000s"]})
#on va en récupérer toutes les lignes nulles
l_nulles_income = df_income.isnull().values.any()
#et les ajouter à L
L+=l_nulles_income
#suppression des lignes de notre liste:
dataset = dataset.drop(index=L)


#les colonnes dont nous devons nous occuper:

#les colonnes à supprimer: on va en récupérer les noms 
#on va récupérer les inutiles et celles qui forment des doublons. Par exemple on a des colonnes dont on a du data 
#categorical et ensuite sa transcription en numerical. On garde alors juste la transcription en numerical
#Les colonnes que l'on enlève et l'explication qui va avec: 
# Les inutiles
#la première: as_of_year, respondent_id,  state_abbr, state_code, hoepa_status, edit_status, sequence_number,
#application_date_indicator
dataset= dataset.drop(columns=["as_of_year", "respondent_id",  "state_abbr", "state_code", "hoepa_status", "edit_status", "sequence_number"])
#et aussi: 
#denial_reason_1, denial_reason_2, denial_reason_3 et rate_spread
#car beaucoup trop de données manquantes
dataset = dataset.drop(columns=["denial_reason_1", "denial_reason_2", "denial_reason_3", "rate_spread"])

#Toutes celles où l'on a la transposition en numerical ensuite:
#On garde toutes les colonnes qui sont en doublons avec les colonnes _name.
#Car on va toutes les traiter avec OneHotEncoder plus simplement
# On identifie toutes les colonnes qui finissent par "_name"
name_cols = [col for col in dataset.columns if col.endswith("_name")]
# On identifie alors toutes les colonnes qui correspondent sont les transpositions
#en numerical des colonnes qui finissent par name. 
radicals = [col[:-5] for col in name_cols]
# On récupère alors les numéros de colonnes auquels correspondent ces colonnes
cols_to_drop = [col for col in dataset.columns if col in radicals]
#Et on enlève toutes ces colonnes
dataset=dataset.drop(columns=cols_to_drop)
#que nous n'ayons plus que des données numerical:
#On applique OneHotEncoder quand il faut. 
"""pour la race de l'applicant notamment: 
On va utiliser oneHotencoder
On ne va pas faire comme dans le dataset. On va créer 6 nouvelles features.
Car on a quand même plus de 500 personnes qui ont plus d'une race et puis cela est toujours bon de savoir comment faire. 
Et à chaque fois pour chaque données on va mettre un 1 ou 0 pour savoir si la donnée a cette race ou non. 
n avait tout coder à la main avant de découvrir one hot encoder. 
On fait pareil pour la race du co applicant. Mais cette fois ci avec 8 catégories. 
On ne va pas faire comme dans le dataset. On va créer 8 nouvelles features.
Car on a quand même plus de 150 personnes qui ont plus d'une race et puis cela est toujours bon de savoir comment faire. 
Et à chaque fois pour chaque données on va mettre un 1 ou 0 pour savoir si la donnée a cette race ou non. 
Et sinon bien sur on met un 1 dans notre provided. """
#On sélectionne les numerical features
numerical_features=dataset.select_dtypes(include='number').columns.tolist()                         # stockage des données numériques (13 colonnes)
#On en déduit les numerical features
categorical_features=list(set(dataset.columns)-set(numerical_features))                             # stockage des données catégoriques(18 colonnes)
#On va maintenant faire le preprocessing: 
#On définit la pipeline de preprocessing
numeric= StandardScaler()
categoric = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
preprocessor = ColumnTransformer(transformers=[("numeric",numeric, numerical_features),
("categoric",categoric, categorical_features)])
#et on l'applique sur les datas
#data preparation des datas categorical
X_categorical=dataset[categorical_features]
#application
X_categorical=preprocessor.fit_transform(X_categorical)
#recréation du dataset total:
dataclean= dataset[numerical_features]+X_categorical
#On montre les datas
dataclean.info()

C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1144: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1149: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\extmath.py:1169: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 54 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   agency_code                     28632 non-null  int64  
 1   loan_amount_000s                28632 non-null  int64  
 2   county_code                     28523 non-null  float64
 3   census_tract_number             28521 non-null  float64
 4   applicant_race_1                28632 non-null  int64  
 5   applicant_race_2                551 non-null    float64
 6   applicant_race_3                49 non-null     float64
 7   applicant_race_4                24 non-null     float64
 8   applicant_race_5                20 non-null     float64
 9   co_applicant_race_1             28632 non-null  int64  
 10  co_applicant_race_2             169 non-null    float64
 11  co_applicant_race_3             17 non-null     float64
 12  co_applicant_race_4             